# Daten bearbeiten

In [2]:
import pandas as pd
import numpy as np

In [62]:
df = pd.read_parquet("2023-03-25_gesamt_ocr_rhein_neckar.parquet")
df = df.rename(columns={'doc_typ': 'doc_typ_old'})

## Dokumenten-Typ berabeiten

In [ ]:
doc_typ = df['doc_typ'].unique()
df_doc_typ = pd.DataFrame(doc_typ)
df_doc_typ.to_csv('doc_typ.csv')

In [ ]:
doc_typ_2 = df[(pd.isnull(df['doc_typ_old']) == True) | (df['doc_typ_old'] == "Sitzungsunterlagen und Beschlüsse")]
doc_typ_2.to_csv('doc_typ_2.csv')

In [64]:
doc_typ_reclass = pd.read_excel("doc_typ_reclass.xlsx")
doc_typ_2_reclass = pd.read_excel("doc_typ_2_reclass.xlsx")

In [65]:
df = df.merge(doc_typ_reclass, on='doc_typ_old', how="left")
df = df.merge(doc_typ_2_reclass, left_index=True, right_on='index', how='left')
df['doc_typ'] = df['doc_typ_x'].combine_first(df['doc_typ_y'])
df.drop(['doc_typ_old','doc_typ_x','doc_typ_y', 'doc_name_y' ,'id', 'index'], axis=1, inplace=True)

In [72]:
df['doc_typ'] = df['doc_typ'].astype('category')
df.to_parquet("2023-04-03_gesamt_rhein_neckar.parquet")

In [79]:
df['doc_typ'].value_counts()

Vorlage             26814
Anhang Vorlage      26152
Beschluss            4738
Niederschrift        4696
Anhang Sitzung       1644
Bekanntmachung       1599
Anfrage              1117
Antrag                888
Anhang Beschluss      777
Name: doc_typ, dtype: int64

## Mobilitätsbezug (ja/nein)

In [76]:
bag_of_words = ["verkehr", "mobilität", 
                "miv","automobil", "pkw", "personenkraftw", "kfz", "kraftfahrzeug", "lkw", "motorrad",
                "stvo", "höchstgeschwindigkeit","führerschein",
                "modal split", "umweltverbund",
                "tankstelle", "ladesäule",
                "ampel", "lsa", "lichtsignalanlage",
                # "stau", "cargo", "logistik",
                "fahrrad", "radverkehr", "schutzstreifen", "radfahr", "abstellanlage", "lastenrad",
                "zu fuß", "fußgäng", "gehweg", "zufußgehen",
                "öpnv", "bus", "bahn", "tram", "haltestelle",
                "car", "bike" ,"roller", "scooter", "sharing",
                "parken", "parkpl", "stellpl", "parkhaus", "garage",
                ]

In [77]:
df['verkehrsbezug'] = df['content'].str.contains('|'.join(bag_of_words), case=False)

In [78]:
df['verkehrsbezug'].value_counts()

False    41486
True     28311
Name: verkehrsbezug, dtype: int64

In [80]:
df.to_parquet("2023-04-03_gesamt_rhein_neckar.parquet")

## Beschlussstatus

In [86]:
import re

In [ ]:
pd.notnull(df['status']).value_counts()

False    52830
True     17253
Name: status, dtype: int64

In [83]:
status = df['status'].unique()
df_status = pd.DataFrame(status)
df_status.to_excel('status.xlsx')

In [96]:
status_list = [r"Beschluss:\s*(\d+\s[-(),\w]+\s*)*",
               "abgelehnt",
               "abgesetzt",
               r"mehrheitlich(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}"
               "Absetzung von der Tagesordnung",
               "einstimmig(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               r"mehrheitlich(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               r"keine Einigung(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               "ohne Abstimmung",
               "nicht in der Tagesordnung aufgenommen",
               "teilweise",
               "Vertagung",
               "vertagt"
               "zur Kenntnis genommen",
               r"zurückgestellt(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               "zurückgewiesen",
               r"Zurückweisung(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,5}",
               "offen"

               ]

status_re = r"(Beschluss(?:[^\wäöüÄÖÜß'-]+[\wäöüÄÖÜß'-]+){0,50})"

In [112]:
no_status = df[pd.isnull(df['status']) == True]

In [120]:
df = df.reset_index()

In [122]:
no_status_filtered = no_status[no_status['doc_typ'] == ('Beschluss' or'Vorlage' or'Antrag'or'Anfrage') ]

In [129]:
df.drop('status_extract', axis=1, inplace=True)

In [132]:
status_extract_all = no_status_filtered['content'].str.extractall(status_re, flags=re.IGNORECASE)

In [126]:
df_status_extract = df[pd.notnull(df['status_extract']) == True]

In [127]:
status_extract_all.to_csv("status_extract_all.csv")

In [128]:
df['content'][23106]

'S T A D T  L A M P E R T H E I M  \nLampertheim, den 24.09.2007 \n \n \n \nB E S C H L U S S  N r .   \n \nder Sitzung der Stadtverordnetenversammlung der Stadt Lampertheim \n \nvom Freitag, den 14.09.2007 um 19:03 Uhr \n \nim Sitzungssaal des Stadthauses, Römerstraße 102, 68623 Lampertheim \n \nEs waren anwesend: \nStass, Brigitte - Stadtverordnetenvorsteherin  \nBauer, Lothar - Stadtverordneter \n \nBerg, Karl-Heinz - Stadtverordneter  \nBittner, Thomas - Stadtverordneter  \nBlepp, Dieter - Stadtverordneter \n \nBrechenser, Dieter - Stadtverordneter \n \nBuschmann, Irma - Stadtverordnete  \nEbert, Rita - Stadtverordnete  \nGalvagno, Nunzio - Stadtverordneter \n \nGötz, Fritz - Stadtverordneter \n \nHahn, Hans - Stadtverordneter \n \nHartmann, Sabine - Stadtverordnete  \nHofmann, Werner - Stadtverordneter \n \nHorstfeld, Karl-Heinz - Stellvertretender Stadtverordnetenvorsteher \n \nHummel, Helmut - Stadtverordneter  \nJacobi, Michael - Stadtverordneter  \nKirsch, Walter - Stadtverord

## Allgemeine Informationen über Datensatz

In [23]:
df = pd.read_parquet("2023-04-03_gesamt_rhein_neckar.parquet")

In [33]:
df['kommune'] = df["kommune"].cat.rename_categories({"Kreis_Heilbronn": "Kreis Heilbronn",
                          "Rhein_Neckar_Kreis": "Rhein-Neckar-Kreis",
                          "Bad Duerkheim": "Bad Dürkheim",
                          })

In [26]:
df = df[df["date"] < "2022-12-31"]

In [29]:
df['date_print'] = df['date'].dt.strftime('%d.%m.%Y')

In [34]:
df.groupby(['kommune']).agg({'date_print': [min, max]}).to_latex("../../latex/Tables/Kommunen_Datum_Min_Max.tex", 
                caption="Zeitspanne der erhobenen Daten",
                label="tab:zeitspanne_daten",
                column_format="lcc"
                )

C:\Users\johan\AppData\Local\Temp\ipykernel_34136\1255178200.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.groupby(['kommune']).agg({'date_print': [min, max]}).to_latex("../../latex/Tables/Kommunen_Datum_Min_Max.tex",
